In [7]:
import numpy as np
from sympy import *
from IPython.display import Markdown, display
import matplotlib.pyplot as plt

This notebook solves the one-dimensional elliptic equation $$\partial_x k(x) \partial_{x} u(x) = f(x)$$ for $x \in \Omega := [\alpha,\beta]$ and boundary conditions $u(\alpha) = u_\alpha$ and $u(\beta) = u_\beta$ using the Galerkin Method

In [9]:
def Galerkin1DElliptic(N, f, alpha, beta, u_alpha, u_beta, conv, u_i):
    
    
    # N = 10             number of spacial points
    # f = lambda x: -1   source function
    # u_alpha            LHS boundary value
    # u_beta             RHS boundary value
    # conv               convective term k(x)


    # problem set up
    Omega = np.linspace(alpha, beta, N)    # spacial domain
    u0 = u_alpha                           # for conviniance
    uN = u_beta                            # for conviniance
    dx = Omega[1] - Omega[0]               # step size
    
    
    # setting orthogonal basis functions
    x, x_km1, x_k = symbols('x x_{k-1} x_k')
    phi_km1 = lambda z, x_km1, x_k: (z - x_km1)/(x_k - x_km1)
    phi_k = lambda z, x_km1, x_k: (x_k - z)/(x_k - x_km1)


    # filling mass matrix
    A_aux = np.zeros((N,N))
    for k in range(N-1):
        M_k = np.zeros((N,N))
        M_k[k,k] = integrate(conv(x), (x, x_km1, x_k)).subs(x_km1,Omega[k]).subs(x_k,Omega[k+1])
        M_k[k,k+1] = -integrate(conv(x), (x, x_km1, x_k)).subs(x_km1,Omega[k]).subs(x_k,Omega[k+1])
        M_k[k+1,k] = -integrate(conv(x), (x, x_km1, x_k)).subs(x_km1,Omega[k]).subs(x_k,Omega[k+1])
        M_k[k+1,k+1] = integrate(conv(x), (x, x_km1, x_k)).subs(x_km1,Omega[k]).subs(x_k,Omega[k+1])
        A_aux += -1/(dx)**2*M_k
    # ------------------------------


    # setting source vector components
    f_km1 = integrate(f(x)*phi_km1(x,x_km1,x_k), (x,x_km1,x_k)) # int_{x_km1}^{x_k} f(x) + phi_{km1}(x) dx
    f_k = integrate(f(x)*phi_k(x,x_km1,x_k), (x,x_km1,x_k)) # int_{x_km1}^{x_k} f(x) + phi_k(x) dx

    # filling source vector
    F_aux = np.zeros(N)
    for k in range(N-1):
        F_k = np.zeros(N)
        F_k[k] = f_km1.subs(x_km1,Omega[k]).subs(x_k,Omega[k+1]) + (abs(u_i[k+1] - u_i[k])/2 + min(u_i[k+1],u_i[k]))
        F_k[k+1] = f_k.subs(x_km1,Omega[k]).subs(x_k,Omega[k+1]) - (abs(u_i[k+1] - u_i[k])/2 + min(u_i[k+1],u_i[k]))
        F_aux += F_k
        #print(abs(u_i[k+1] - u_i[k])/2)
    # ------------------------------


    

    # dirichlet boundary conditions
    A = A_aux[1:-1,1:-1]
    F = F_aux[1:-1]
    F[0] += -A_aux[1,0]*u0
    F[-1] += -A_aux[-2, -1]*uN
    # ------------------------------



    # constructing solution
    u_aux = np.linalg.solve(A,F)
    u = np.zeros(N)
    u[0] = u0
    u[-1] = uN
    u[1:-1] = u_aux
    
    return u